In [41]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown, IntSlider, FloatSlider, VBox, interactive_output
from scipy.special import genlaguerre
from scipy.special import jv, yn, iv, kv


def hermite_gaussian(m, n, x, y, w0):
    Hm = np.polynomial.hermite.hermval(x / w0, [0] * m + [1])  # Hermite polynomial of degree m
    Hn = np.polynomial.hermite.hermval(y / w0, [0] * n + [1])  # Hermite polynomial of degree n
    envelope = np.exp(-(x**2 + y**2) / w0**2)  # Gaussian envelope
    intensity = (envelope * Hm * Hn) ** 2
    return intensity

def laguerre_gaussian(p, l, r, phi, w0):
    radial = genlaguerre(p, np.abs(l))(2 * (r / w0)**2)
    azimuthal = np.exp(1j * l * phi)
    envelope = np.exp(-(r / w0)**2)
    intensity = (envelope * r**np.abs(l) * radial * azimuthal).real**2
    return intensity

def bessel_gaussian(m, r, radial_periodicity, w0):
    J = jv(m, 2 * np.pi * r / radial_periodicity)
    envelope = np.exp(-(r**2) / w0**2)
    intensity = (J * envelope) ** 2
    return intensity



In [42]:
def plot_gaussian(mode_type, mode1, mode2, beam_waist, radial_periodicity):
    x = np.linspace(-5, 5, 500)
    y = np.linspace(-5, 5, 500)
    X, Y = np.meshgrid(x, y)
    R = np.sqrt(X**2 + Y**2)
    Phi = np.arctan2(Y, X)

    if mode_type == "Hermite-Gaussian":
        intensity = hermite_gaussian(mode1, mode2, X, Y, beam_waist)
    elif mode_type == "Laguerre-Gaussian":
        intensity = laguerre_gaussian(mode1, mode2, R, Phi, beam_waist)
    elif mode_type == "Bessel-Gaussian":
        intensity = bessel_gaussian(mode1, R, radial_periodicity, beam_waist)
    else:
        intensity = np.exp(-(X**2 + Y**2) / beam_waist**2)  # TEM00

    plt.figure(figsize=(8, 6))
    plt.imshow(intensity, extent=(-5, 5, -5, 5), origin="lower", cmap="viridis")
    plt.colorbar(label="Intensity")
    plt.title(f"{mode_type} Mode ({mode1}, {mode2})")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()

# Interactive widget
def interactive_plot():
    interact(
        plot_gaussian,
        mode_type=Dropdown(
            options=["Hermite-Gaussian", "Laguerre-Gaussian", "Bessel-Gaussian", "TEM00"],
            value="Hermite-Gaussian",
            description="Beam Type",
        ),
        mode1=IntSlider(min=0, max=10, value=0, description="Mode 1"),
        mode2=IntSlider(min=0, max=10, value=0, description="Mode 2"),
        beam_waist=FloatSlider(min=0.5, max=5, step=0.1, value=1.0, description="Beam Waist"),
        radial_periodicity=FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description="Radial Periodicity (Lambda):"),
    )

interactive_plot()

interactive(children=(Dropdown(description='Beam Type', options=('Hermite-Gaussian', 'Laguerre-Gaussian', 'Bes…